In [ ]:
%pylab inline
from scipy.interpolate import interpn
from helpFunctions import surfacePlot
import numpy as np
from multiprocessing import Pool
from functools import partial
import warnings
import math
warnings.filterwarnings("ignore")
np.printoptions(precision=2)

In [ ]:
# time line
T_min = 0
T_max = 70
T_R = 45
# discounting factor
beta = 1/(1+0.02)
# utility function parameter 
gamma = 2
# relative importance of housing consumption and non durable consumption 
alpha = 0.8
# parameter used to calculate the housing consumption 
kappa = 0.3
# depreciation parameter 
delta = 0.025
# housing parameter 
chi = 0.3
# uB associated parameter
B = 2
# # minimum consumption 
# c_bar = 3
# constant cost 
c_h = 0.5
# All the money amount are denoted in thousand dollars
earningShock = [0.8,1.2]
# Define transition matrix of economical states
# GOOD -> GOOD 0.8, BAD -> BAD 0.6
Ps = np.array([[0.6, 0.4],[0.2, 0.8]])
# current risk free interest rate
# r_b = np.array([0.01 ,0.03]) modify the risk free return 
r_b = np.array([0.03, 0.05])
# stock return depends on current and future econ states
# r_k = np.array([[-0.2, 0.15],[-0.15, 0.2]])
r_k = np.array([[-0.15, 0.20],[-0.15, 0.20]])
# expected return on stock market
# r_bar = 0.0667
r_bar = 0.02
# probability of survival
Pa = np.load("prob.npy")
# deterministic income
detEarning = np.load("detEarning.npy")
# probability of employment transition Pe[s, s_next, e, e_next]
Pe = np.array([[[[0.3, 0.7], [0.1, 0.9]], [[0.25, 0.75], [0.05, 0.95]]],
               [[[0.25, 0.75], [0.05, 0.95]], [[0.2, 0.8], [0.01, 0.99]]]])
# tax rate before and after retirement
tau_L = 0.2
tau_R = 0.1
# constant state variables: Purchase value 250k, down payment 50k, mortgage 200k, interest rate 3.6%,
# 55 payment period, 8.4k per period. One housing unit is roughly 1 square feet. Housing price 0.25k/sf 

# some variables associate with 401k amount
Nt = [np.sum(Pa[t:]) for t in range(T_max-T_min)]
Dt = [np.ceil(((1+r_bar)**N - 1)/(r_bar*(1+r_bar)**N)) for N in Nt]
# mortgate rate 
rh = 0.036
D = [((1+rh)**N - 1)/(rh*(1+rh)**N) for N in range(T_max-T_min)]
# income fraction goes into 401k 
yi = 0.005
# housing unit
H = 750
# housing price constant 
pt = 250/1000
# 30k rent 1000 sf
pr = 30/1000

# wealth discretization 
ws = np.array([10,25,50,75,100,125,150,175,200,250,500,750,1000,1500,3000])
w_grid_size = len(ws)
# 401k amount discretization 
ns = np.array([1, 5, 10, 15, 25, 40, 65, 100, 150, 300, 400,1000])
n_grid_size = len(ns)
# Mortgage amount, * 0.25 is the housing price per unit
Ms = np.array([0.01*H,0.05*H,0.1*H,0.2*H,0.3*H,0.4*H,0.5*H,0.6*H,0.7*H,0.8*H]) * pt
M_grid_size = len(Ms)
# Improvement amount 
gs = np.array([0,25,50,75,100])
g_grid_size = len(gs)

points = (ws,ns,Ms,gs)

In [ ]:
import quantecon as qe
import timeit
mc = qe.MarkovChain(Ps)

#Vgrid = np.load("Vgrid_i.npy")
cgrid = np.load("cgrid" + str(H)+ ".npy")
bgrid = np.load("bgrid" + str(H)+ ".npy")
kgrid = np.load("kgrid" + str(H)+ ".npy")
igrid = np.load("igrid" + str(H)+ ".npy")
qgrid = np.load("qgrid" + str(H)+ ".npy")

def action(t, x):
    w, n, M, g_lag, e, s = x
    c = interpn(points, cgrid[:,:,:,:,e,s,t], x[:4], method = "nearest", bounds_error = False, fill_value = None)[0]
    b = interpn(points, bgrid[:,:,:,:,e,s,t], x[:4], method = "nearest", bounds_error = False, fill_value = None)[0]
    k = interpn(points, kgrid[:,:,:,:,e,s,t], x[:4], method = "nearest", bounds_error = False, fill_value = None)[0]
    i = interpn(points, igrid[:,:,:,:,e,s,t], x[:4], method = "nearest", bounds_error = False, fill_value = None)[0]
    q = interpn(points, qgrid[:,:,:,:,e,s,t], x[:4], method = "nearest", bounds_error = False, fill_value = None)[0]
    return (c,b,k,i,q)

#Define the earning function, which applies for both employment and unemployment, good econ state and bad econ state 
def y(t, x):
    w, n, M, g_lag, e, s = x
    if t <= T_R:
        welfare = 5
        return detEarning[t] * earningShock[int(s)] * e + (1-e) * welfare
    else:
        return detEarning[t]

#Define the evolution of the amount in 401k account 
def gn(t, n, x, s_next):
    w, n, M, g_lag, e, s = x
    if t <= T_R and e == 1:
        # if the person is employed, then 5 percent of his income goes into 401k 
        # i = 0.05
        n_cur = n + y(t, x) * yi
    elif t <= T_R and e == 0:
        # if the perons is unemployed, then n does not change 
        n_cur = n
    else:
        # t > T_R, n/discounting amount will be withdraw from the 401k 
        n_cur = n - n/Dt[t]
    return (1+r_k[int(s), s_next])*n_cur 

In [ ]:
Vgrid = np.load("Vgrid_renting.npy") 
cgrid = np.load("cgrid_renting.npy") 
bgrid = np.load("bgrid_renting.npy") 
kgrid = np.load("kgrid_renting.npy") 
hgrid = np.load("hgrid_renting.npy") 
Mgrid = np.load("Mgrid_renting.npy")